#*importing data using pandas*

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt,exp,pi
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
data=pd.read_csv('diabetes.csv')                               # In diabetes column 1-represent person suffering with diabetes
data=data.drop(["Pregnancies","SkinThickness","DiabetesPedigreeFunction"],axis=1) # 0- represent person not suffering with diabetes
data.head(10)

,Glucose,BloodPressure,Insulin,BMI,Age,Outcome
0,148,72,0,33.6,50,1
1,85,66,0,26.6,31,0
2,183,64,0,23.3,32,1
3,89,66,94,28.1,21,0
4,137,40,168,43.1,33,1
5,116,74,0,25.6,30,0
6,78,50,88,31.0,26,1
7,115,0,0,35.3,29,0
8,197,70,543,30.5,53,1
9,125,96,0,0.0,54,1


In [28]:
data.describe()# knowing about data set 

,Glucose,BloodPressure,Insulin,BMI,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,120.894531,69.105469,79.799479,31.992578,33.240885,0.348958
std,31.972618,19.355807,115.244002,7.884160,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
25%,99.000000,62.000000,0.000000,27.300000,24.000000,0.000000
50%,117.000000,72.000000,30.500000,32.000000,29.000000,0.000000
75%,140.250000,80.000000,127.250000,36.600000,41.000000,1.000000
max,199.000000,122.000000,846.000000,67.100000,81.000000,1.000000


In [29]:
print(data.ndim)
print(data.shape)
print(data.size)
print(data["Outcome"].value_counts())# finding data is balanced or not

2
(768, 6)
4608
Outcome
0    500
1    268
Name: count, dtype: int64


In [33]:
data["Glucose"].describe()

count    768.000000
mean     120.894531
std       31.972618
min        0.000000
25%       99.000000
50%      117.000000
75%      140.250000
max      199.000000
Name: Glucose, dtype: float64

In [32]:
data["BloodPressure"].describe()

count    768.000000
mean      69.105469
std       19.355807
min        0.000000
25%       62.000000
50%       72.000000
75%       80.000000
max      122.000000
Name: BloodPressure, dtype: float64

In [35]:
data["Age"].describe()

count    768.000000
mean      33.240885
std       11.760232
min       21.000000
25%       24.000000
50%       29.000000
75%       41.000000
max       81.000000
Name: Age, dtype: float64

In [34]:
X=data.iloc[:,:5].values
Y=data.iloc[:,5:6].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=5,shuffle=True)
#  no of rows in data set=768
#  no of rows in train_data=537
#  no of rows in test_data=231

## *training the model*

In [35]:

def mean_of(col,tar):
    sum=c=0
    r=len(X_train)
    for i in range(r):
        if(Y_train[i][0]==tar):
            sum+=X_train[i][col]
            c+=1
    return sum/c

def var(col,tar,m):
    v=c=0
    r=len(X_train)
    for i in range(r):
        if(Y_train[i][0]==tar):
            t=(X_train[i][col]-m)
            v+=t*t
            c+=1
    return v/(c-1)

d={1:{"Glucose":{},"BloodPressure":{},"Insulin":{},"BMI":{},"Age":{}},0:{"Glucose":{},"BloodPressure":{},"Insulin":{},"BMI":{},"Age":{}}}
d[1]["Glucose"]["mean"]=mean_of(0,1)
d[1]["Glucose"]["vari"]=var(0,1,d[1]["Glucose"]["mean"])

d[1]["BloodPressure"]["mean"]=mean_of(1,1)
d[1]["BloodPressure"]["vari"]=var(1,1,d[1]["BloodPressure"]["mean"])

d[1]["Insulin"]["mean"]=mean_of(2,1)
d[1]["Insulin"]["vari"]=var(2,1,d[1]["Insulin"]["mean"])

d[1]["BMI"]["mean"]=mean_of(3,1)
d[1]["BMI"]["vari"]=var(3,1,d[1]["BMI"]["mean"])

d[1]["Age"]["mean"]=mean_of(4,1)
d[1]["Age"]["vari"]=var(4,1,d[1]["Age"]["mean"])

d[0]["Glucose"]["mean"]=mean_of(0,0)
d[0]["Glucose"]["vari"]=var(0,0,d[0]["Glucose"]["mean"])

d[0]["BloodPressure"]["mean"]=mean_of(1,0)
d[0]["BloodPressure"]["vari"]=var(1,0,d[0]["BloodPressure"]["mean"])

d[0]["Insulin"]["mean"]=mean_of(2,0)
d[0]["Insulin"]["vari"]=var(2,0,d[0]["Insulin"]["mean"])

d[0]["BMI"]["mean"]=mean_of(3,0)
d[0]["BMI"]["vari"]=var(3,0,d[0]["BMI"]["mean"])

d[0]["Age"]["mean"]=mean_of(4,0)
d[0]["Age"]["vari"]=var(4,0,d[0]["Age"]["mean"])

#prior probabilities
prob_of_1=sum([1 for i in range(len(Y_train)) if Y_train[i][0]==1])/len(Y_train)
prob_of_0=1-prob_of_1

## *Testing the Model*

In [36]:
def pdf(mu,var,x):
    t=x-mu
    t=t*t
    t=t/(2*var)
    t=exp(-t)
    return t/(2*pi*var)
    
predi_list=[]
n=len(X_test)
for i in range(n):
    Pgl_1=pdf(d[1]["Glucose"]["mean"],d[1]["Glucose"]["vari"],X_test[i][0])
    Pbp_1=pdf(d[1]["BloodPressure"]["mean"],d[1]["BloodPressure"]["vari"],X_test[i][1])
    Pin_1=pdf(d[1]["Insulin"]["mean"],d[1]["Insulin"]["vari"],X_test[i][2])
    Pbm_1=pdf(d[1]["BMI"]["mean"],d[1]["BMI"]["vari"],X_test[i][3])
    Pag_1=pdf(d[1]["Age"]["mean"],d[1]["Age"]["vari"],X_test[i][4])
    P_1=(Pgl_1 *Pbp_1 *Pin_1 *Pbm_1 *Pag_1)*(prob_of_1)

    Pgl_0=pdf(d[0]["Glucose"]["mean"],d[0]["Glucose"]["vari"],X_test[i][0])
    Pbp_0=pdf(d[0]["BloodPressure"]["mean"],d[0]["BloodPressure"]["vari"],X_test[i][1])
    Pin_0=pdf(d[0]["Insulin"]["mean"],d[0]["Insulin"]["vari"],X_test[i][2])
    Pbm_0=pdf(d[0]["BMI"]["mean"],d[0]["BMI"]["vari"],X_test[i][3])
    Pag_0=pdf(d[0]["Age"]["mean"],d[0]["Age"]["vari"],X_test[i][4])
    P_0=(Pgl_0 *Pbp_0 *Pin_0 *Pbm_0 *Pag_0)*(prob_of_0)

    if(P_1 > P_0):
        predi_list.append(1)
    else:
        predi_list.append(0)

    

## *finding accuracy of model*

In [37]:
n=len(Y_test)
c=0
for i in range(n):
    if(Y_test[i][0]==predi_list[i]):
        c+=1
print("Accuracy of model:"+str(c/n))

Accuracy of model:0.7835497835497836
